# Building and Submitting Argo Workflows Using Hera

In [1]:
from hera.workflows import Steps, Step, Workflow, script, WorkflowsService
from hera.workflows.models import WorkflowTemplateRef, TemplateRef
from hera.shared import global_config

In [2]:
# token is obtained from https://workflows.argo.cuahsi.io/userinfo
argo_token = "v2:eyJhbGciOiJSU0EtT0FFUC0yNTYiLCJlbmMiOiJBMjU2R0NNIiwiemlwIjoiREVGIn0.Jmwv5MWyI9zAt3zM5teLzoXAu3VbTicO9I-HtPdI88nRp_ETGXkFQqC5aM_msNYlJSF5ncrugC5RexrhvxPIq5tZeWHDG9dzif6TDn1Z6G8vuicIcbwKCbhtveFWoJHUNO4rz714Rt5decdPeIwxBTJ2bKoPnK8HZtrit2fDseJOrD9EYBUSpjMZt-aiSGD7SuHye08kNggmHVEzfNOx-ehXI2doq62_bqZNNOOLR4H_p48SV2DVaQHU28yH5dVKBPoS9gQcxM0UTeadyjNm4M_I4s4-g-8tVDhzadUih6rtFw33smvK_gjon30N4mtUP2rNrhLjGy701bbj_RnFRA.7RZ66x0RfHrFzmBA.0ntF913jQdh3Urda5EOMqMI2Gzn59ttOcmfmX3dfvkLzSjktGK6UTcQE7d3lLRPejzeOrfe0LfFVjVcJDFvkQoITZIHhBtc.Q81RRcNnwtUKQ0y5hknjiw"
argo_host  = "https://workflows.argo.cuahsi.io"
argo_ns    = "workflows"

In [3]:
ws=WorkflowsService(host=argo_host,
                   token=argo_token,
                   namespace=argo_ns)

Test the connection

In [4]:
ws.get_info()

InfoResponse(links=None, managed_namespace=None, modals=None, nav_color=None)

Define a function to print job status

In [52]:
import time
import datetime 

def display_status(workflow):
    job_name = workflow.name

    # initial values
    finished_at = None
    start_time = None
    
    print(f'Job: {job_name}') 
    while finished_at is None:
        # query the job status
        status = workflow.workflows_service.get_workflow(job_name).status

        # set start time
        if start_time is None:
            start_time = status.started_at.__root__

        # get current time
        now = datetime.datetime.now(datetime.timezone.utc)

        # display progress
        print(f'Progress: {status.progress} ' +
              f'[{status.phase}]' + 
              f' - Elapsed time={now-start_time} seconds', end='\r')

        # update finished time
        finished_at = status.finished_at
    
        time.sleep(1)
        
    print('')
    print(f'Finished at: {str(finished_at.__root__)}')

## Run a sample job

Create a simple job (https://github.com/argoproj-labs/hera#examples) 

In [53]:
@script()
def echo(message: str):
    print(message)
    
with Workflow(
    generate_name="hello-world-",
    entrypoint="steps",
    namespace="workflows",
    workflows_service=ws
) as w:
    with Steps(name="steps"):
        
        echo(arguments={"message": "Hello world!"})


Display the YAML that was generated

In [54]:
print(w.to_yaml())

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: hello-world-
  namespace: workflows
spec:
  entrypoint: steps
  templates:
  - name: steps
    steps:
    - - arguments:
          parameters:
          - name: message
            value: Hello world!
        name: echo
        template: echo
  - inputs:
      parameters:
      - name: message
    name: echo
    script:
      command:
      - python
      image: python:3.8
      source: 'import os

        import sys

        sys.path.append(os.getcwd())

        import json

        try: message = json.loads(r''''''{{inputs.parameters.message}}'''''')

        except: message = r''''''{{inputs.parameters.message}}''''''


        print(message)'



Submit the job

In [55]:
_ = w.create()

Display the job status

In [56]:
display_status(w)

Job: hello-world-m8c8l
Job progress: 1/1 [Succeeded] - Elapsed time=0:00:11.871370 seconds
Job finished at: 2023-07-20 14:15:12+00:00


## Run a Job for a Pre-defined Template

List workflow templates that exist in our cluster.

In [57]:
for wt in ws.list_workflow_templates().items:
    print (wt.metadata.name)

linked-ngen-metadata
metadata-extractor
ngen-subset-v1
parflow-subset
parflow-v1-plus-extract
serenity-now
test-input-artifacts


Create a workflow template reference from one of the templates listed above

In [58]:
wt_ref = WorkflowTemplateRef(name="metadata-extractor", cluster_scope=True)

Create a new workflow using this template

In [63]:
with Workflow(
    generate_name="template-test-",
    entrypoint="tasks",
    workflows_service=ws,
    namespace='workflows',
) as w:
    metadata_extract_ref = TemplateRef(
        name="metadata-extractor",
        template="metadata-extractor",
    )
    with Steps(name="tasks") as s:
        Step(
            name='call-metadata-extract',
            template_ref=metadata_extract_ref,
            arguments={'bucket': 'subsetter-outputs',
                        'path': 'ngen/abc123',
                       },
            )

Display the YAML that was created.

In [64]:
print(w.to_yaml())

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: template-test-
  namespace: workflows
spec:
  entrypoint: tasks
  templates:
  - name: tasks
    steps:
    - - arguments:
          parameters:
          - name: bucket
            value: subsetter-outputs
          - name: path
            value: ngen/abc123
        name: call-metadata-extract
        templateRef:
          name: metadata-extractor
          template: metadata-extractor



Submit the job

In [65]:
_ = w.create()

Display the job status

In [66]:
display_status(w)

Job: template-test-64lsf
Job progress: 1/1 [Succeeded] - Elapsed time=0:00:11.199767 seconds
Job finished at: 2023-07-20 14:18:21+00:00
